In [1]:
#!pip install -q seaborn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


In [2]:
dictoo = {}
thesetofdata = ()
listofcolumns = []



dictoo["x2"] = np.random.rand(1000)

dictoo["x"] = 10 + 20*np.random.rand(1000)

#dictoo["x^2"] =  dictoo["x"]  **2
#dictoo["x^3"] =  dictoo["x"] **3
#dictoo["1/x"] =  1/ dictoo["x"] 
#dictoo["sinx"] =  np.sin(dictoo["x"] )
#dictoo["cosx"] = np.cos(dictoo["x"] )

dictoo["y"] = dictoo["x2"]**2 + dictoo["x"]






In [3]:
for i in dictoo.keys():
  listofcolumns.append(i)

thesetofdata = dictoo.values()

dataset = pd.DataFrame(data=thesetofdata)
dataset = dataset.transpose()
dataset.columns = listofcolumns
renameedict = {listofcolumns[0]:"input", listofcolumns[-1]:"result"}
dataset = dataset.rename(columns=renameedict)

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
#sns.pairplot(train_dataset, diag_kind='kde')
#train_dataset.describe().transpose()
train_features = train_dataset.copy()
test_features = test_dataset.copy()
train_labels = train_features.pop('result')
test_labels = test_features.pop('result')
train_dataset.describe().transpose()[['mean', 'std']]


normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))
singlevariable = np.array(train_features['input'])
singlevariable_normalizer = preprocessing.Normalization(input_shape=[1,])
singlevariable_normalizer.adapt(singlevariable)
singlevariable_model = tf.keras.Sequential([singlevariable_normalizer,layers.Dense(units=1)])
#singlevariable_model.summary()
#singlevariable_model.predict(singlevariable[:10])
singlevariable_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1),loss='mean_absolute_error')
history = singlevariable_model.fit(train_features['input'], train_labels, verbose=0, epochs=100,validation_split = 0.2)
# singlevariable_model.predict(singlevariable[:10])
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
print(hist.tail())
#def plot_loss(history):
#  plt.plot(history.history['loss'], label='loss')
#  plt.plot(history.history['val_loss'], label='val_loss')
#  plt.ylim([0, 10])
#  plt.xlabel('Epoch')
#  plt.ylabel('Error [result]')
#  plt.legend()
#  plt.grid(True)
#plot_loss(history)

# test_results = {}
# 
# test_results['singlevariable_model'] = singlevariable_model.evaluate(
#     test_features['input'],
#     test_labels, verbose=0)
# 
# x = tf.linspace(0.0, 1, 10)
# y = singlevariable_model.predict(x)
# 
# def plot_singlevariable(x, y):
#   plt.scatter(train_features['input'], train_labels, label='Data')
#   plt.plot(x, y, color='k', label='Predictions')
#   plt.xlabel('input')
#   plt.ylabel('result')
#   plt.legend()
# 
# 
# plot_singlevariable(x,y)



linear_model = tf.keras.Sequential([normalizer, layers.Dense(units=1)])

#linear_model.layers[1].kernel

linear_model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1),loss='mean_absolute_error')

history = linear_model.fit(
    train_features, train_labels, 
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

#plot_loss(history)
#test_results['linear_model'] = linear_model.evaluate(test_features, test_labels, verbose=0)
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
print(hist.tail())


def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

dnn_singlevariable_model = build_and_compile_model(singlevariable_normalizer)
#dnn_singlevariable_model.summary()

history = dnn_singlevariable_model.fit(
    train_features['input'], train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

#plot_loss(history)

#x = tf.linspace(0.0, 250, 251)
#y = dnn_singlevariable_model.predict(x)
#plot_singlevariable(x, y)

#test_results['dnn_singlevariable_model'] = dnn_singlevariable_model.evaluate(
#    test_features['result'], test_labels,
#    verbose=0)


hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
print(hist.tail())

dnn_model = build_and_compile_model(normalizer)
#dnn_model.summary()


history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

#plot_loss(history)
#test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)
#pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T
#
#test_predictions = dnn_model.predict(test_features).flatten()
#
#a = plt.axes(aspect='equal')
#plt.scatter(test_labels, test_predictions)
#plt.xlabel('True Values [MPG]')
#plt.ylabel('Predictions [MPG]')
#lims = [0, 50]
#plt.xlim(lims)
#plt.ylim(lims)
#_ = plt.plot(lims, lims)
#
#error = test_predictions - test_labels
#plt.hist(error, bins=25)
#plt.xlabel('Prediction Error [MPG]')
#_ = plt.ylabel('Count')

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
print(hist.tail())


        loss  val_loss  epoch
95  5.156955  5.450515     95
96  5.184183  5.347929     96
97  5.154091  5.487887     97
98  5.265382  5.418309     98
99  5.159566  5.536500     99
        loss  val_loss  epoch
95  0.065368  0.084601     95
96  0.062945  0.064616     96
97  0.063912  0.086809     97
98  0.064824  0.080122     98
99  0.061711  0.065190     99
        loss  val_loss  epoch
95  5.122212  5.309550     95
96  5.116092  5.308816     96
97  5.198895  5.363797     97
98  5.162922  5.325118     98
99  5.107795  5.295657     99
        loss  val_loss  epoch
95  0.021938  0.025670     95
96  0.025304  0.021824     96
97  0.027339  0.027454     97
98  0.028432  0.042834     98
99  0.029761  0.020880     99
